In [1]:
import random

import datasets
import torch
from langchain import hub
from transformers import pipeline

In [2]:
df = datasets.load_dataset('kreimben/leetcode_with_youtube_captions')['train']
df = df.to_pandas()
df.sample(3)

,cc_content,id,thumbnail,title_x,question_content,java,c++,python,javascript,title_y,tag,level,success_rate,total_submission,total_accepted,question_likes,question_dislikes,question_hints,similar_question_ids
11457,hey so welcome back and there's another hey s...,377,https://i.ytimg.com/vi/hg_U5GVEM-k/maxresdefau...,Combination Sum IV,Given an array of **distinct** integers `nums`...,\n ```java\npublic int combinationSum(int[]...,\n ```cpp\n#include <vector>\nusing namespa...,"\n ```python\ndef combination_sum(nums, tar...",\n ```javascript\nfunction combinationSum(n...,Combination Sum IV,"Array,Dynamic Programming",Medium,49.3,482241.0,237874.0,3333.0,383.0,None,39
15257,hello everyone let's try to be a cheat hello ...,515,https://i.ytimg.com/vi/SsH9p1h7Ca8/maxresdefau...,Find Largest Value in Each Tree Row,"Given the `root` of a binary tree, return _an ...",\n ```java\nimport java.util.*;\n\nclass Tr...,\n ```cpp\n#include <vector>\n#include <que...,\n ```python\nfrom collections import deque...,\n ```javascript\nclass TreeNode {\n con...,Find Largest Value in Each Tree Row,"Tree,Depth-First Search,Breadth-First Search,B...",Medium,64.4,291567.0,187850.0,1980.0,82.0,None,None
769,hey folks welcome back to another video hey f...,284,https://i.ytimg.com/vi/Q1fFspHfiZ8/sddefault.jpg,Peeking Iterator,Design an iterator that supports the `peek` op...,\n ```java\nimport java.util.Iterator;\n\nc...,\n ```cpp\n#include <iterator>\n\nclass Pee...,\n ```python\nclass PeekingIterator:\n d...,\n ```javascript\nclass PeekingIterator {\n...,Peeking Iterator,"Array,Design,Iterator",Medium,53.6,291298.0,156171.0,994.0,685.0,"Think of ""looking ahead"". You want to cache th...","173,251,281"


In [3]:
N = len(df)
idx = random.randint(0, N)

example = df.loc[idx, 'cc_content']

In [ ]:
# Load the HF pipeline using Gemma 2B from Kaggle
pipe = pipeline(
    "text-generation",
    model='google/gemma-2b',
    model_kwargs={"torch_dtype": torch.float16},
    device='cuda',
    max_new_tokens=1024
)

In [ ]:
messages = [
    {
        "role": "user",
        "content": """
        Summarize the following text in a technical way and approaching to interprete the coding test question. 
        Focus on facts, numbers and strategies used. 
        Divide the summary in chapters, be impersonal and use bullet points:
        
        {}""".format(example)
    }
]

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(
    prompt,
    do_sample=True,
    temperature=0.1,
    top_k=20,
    top_p=0.3,
    add_special_tokens=True
)
res = outputs[0]["generated_text"][len(prompt):]